### Environment

In [37]:
import arcpy
from collections import defaultdict

arcpy.env.workspace = r"D:\sem5\pag\lab3\lab3\lab3.gdb"
arcpy.env.overwriteOutput = True

In [ ]:
# Merge the two road shapefiles
arcpy.management.Merge(
    inputs="L4_2_BDOT10k__OT_SKJZ_L;L4_1_BDOT10k__OT_SKJZ_L",
    output=r"D:\sem5\pag\lab3\lab3\lab3.gdb\SKJZ",
    field_mappings='ID1 "ID1" true true false 10 Long 0 0,First,#,L4_2_BDOT10k__OT_SKJZ_L,ID1,-1,-1,L4_1_BDOT10k__OT_SKJZ_L,ID1,-1,-1',
    add_source="NO_SOURCE_INFO"
)

### Define the start and end vertex

In [46]:
start = 1313
end = 1928

# Make graph from a shapefile
This code takes a shapefile file containing a road network and processes it producing a vertcies and edges file. \
vertex: id, x, y, edges \
edge: id, from, to, road_id, length

In [16]:
roads_shp = "SKJZ"

vertices_file = "vertices.txt"
edges_file = "edges.txt"

vertices = {}
edges = []

# Create vertex if one doesn't exist with the same coords
def add_vertex(x, y, vertex_id):
    if (x, y) not in vertices:
        vertices[(x, y)] = {'id': vertex_id, 'x': x, 'y': y, 'edges': []}
    return vertices[(x, y)]['id']

# Read road shp
with arcpy.da.SearchCursor(roads_shp, ["SHAPE@", "OBJECTID"]) as cursor:
    vertex_id = 0
    edge_id = 0

    for row in cursor:
        line = row[0]
        FID = row[1]
        
        # eead start and end point, round to int
        start_point = line.firstPoint
        end_point = line.lastPoint

        start_point = (round(start_point.X, 0), round(start_point.Y, 0))
        end_point = (round(end_point.X, 0), round(end_point.Y, 0))
        
        start_vertex = add_vertex(start_point[0], start_point[1], vertex_id)
        vertex_id += 1 if start_vertex == vertex_id else 0

        end_vertex = add_vertex(end_point[0], end_point[1], vertex_id)
        vertex_id += 1 if end_vertex == vertex_id else 0

        # add edges
        edges.append({
            "id": edge_id,
            "from": start_vertex,
            "to": end_vertex,
            "road_id": FID,
            "length": int(line.length)
        })

        edge_id += 1
        
        # add edges to the vertex file
        vertices[start_point]['edges'].append(edge_id)
        vertices[end_point]['edges'].append(edge_id)


# Write vertices
with open(vertices_file, "w") as vf:
    vf.write("id\tx\ty\tedges\n")
    for v in vertices.values():
        vf.write(f"{v['id']}\t{v['x']}\t{v['y']}\t{','.join(map(str, v['edges']))}\n")

# Write edges
with open(edges_file, "w") as ef:
    ef.write("id\tfrom\tto\troad_id\tlength\n")
    for edge in edges:
        ef.write(f"{edge['id']}\t{edge['from']}\t{edge['to']}\t{edge['road_id']}\t{edge['length']}\n")

# print("Vertex count:", len(vertices))
# print("Edge count:", len(edges))


Liczba wierzchołków: 31058
Liczba krawędzi: 38559


<Result 'D:\\sem5\\pag\\lab3\\lab3\\lab3.gdb\\vertices'>

### Display all vertices 

In [17]:
# Makes points from the vertex.txt file
arcpy.management.XYTableToPoint(
    in_table="vertices.txt",
    out_feature_class=r"vertices",
    x_field="x",
    y_field="y",
    z_field=None,
    coordinate_system='PROJCS["ETRS_1989_UWPP_1992",GEOGCS["GCS_ETRS_1989",DATUM["D_ETRS_1989",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Gauss_Kruger"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",-5300000.0],PARAMETER["Central_Meridian",19.0],PARAMETER["Scale_Factor",0.9993],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]];-5119200 -15295100 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision'
)

<Result 'D:\\sem5\\pag\\lab3\\lab3\\lab3.gdb\\vertices'>

# BFS path finder

In [47]:
path_file = r"path.txt"
vertices = r"vertices.txt"
edges = r"edges.txt"


visited_vertices = r"visited_vertices.txt"
roads_visited = r"roads_visited.txt"

# read undirected graph
def read_graph_undirected(filename):
    f = open(filename)
    g = defaultdict(list)
    road_ids = {}
    f.readline()
    for line in f:
        e = [int(x) for x in line.split()]
        g[e[1]].append(e[2])
        g[e[2]].append(e[1])
        road_id = e[3]

        road_ids[(e[1], e[2])] = road_id
        road_ids[(e[2], e[1])] = road_id
        
    return g, road_ids

# retrives path from a to b
def retrieve_path(prev, a, b, road_ids):
    path = [b]
    road_path = []
    while b != a:
        b = prev[b]
        path.append(b)
        road_path.append(road_ids[(b, path[-2])])
    path.reverse()
    road_path.reverse()
    return path, road_path

# Finds the shortest path in terms of number of edges - BFS
def bfs_path(graph,road_ids, a, b):
    # init
    queue = []
    visited = set()
    prev = {}

    # add staring vertex to the queue
    queue.append(a)
    visited.add(a)
    prev[a] = None

    while queue:
        u = queue.pop(0)                    # dequeue the first vertex
        if u == b:
            return retrieve_path(prev, a, b, road_ids) # path found?    

        for w in graph[u]:                  # loop over all neighbours
            if w not in visited:            # not visited yet
                queue.append(w)             # enqueue them
                visited.add(w)
                prev[w] = u
    
    return None


g, road_ids = read_graph_undirected(edges)
path, road_path = bfs_path(g, road_ids, start, end)

print(path)
print(road_path)


# find the vertices in the path and write the coords to the file
with open(vertices, 'r') as f:
    vertices = f.readlines()
    vertices = [x.strip() for x in vertices]
    path = [vertices[i+2].split()[0] for i in path]

# write the path to a file
with open(visited_vertices, 'w') as f:
    f.write("id\tx\ty\tedges\n")
    for i in path:
        f.write(vertices[int(i)] + '\n')

# write the road path to a file
with open(roads_visited, 'w') as f:
    f.write("id\tx\ty\tedges\n")
    for i in road_path:
        f.write(str(i) + '\n')

def create_sql_expression(road_path):
    sql_expression = "OBJECTID = " + str(road_path[0])
    for i in road_path[1:]:
        sql_expression += " Or OBJECTID = " + str(i)

    return sql_expression

sql_expression = create_sql_expression(road_path)


[1313, 1214, 1215, 1775, 1827, 1928]
[1261, 1189, 1981, 1983, 2122]


### Display the vertices on the path

In [48]:
arcpy.management.XYTableToPoint(
    in_table="visited_vertices.txt",
    out_feature_class=r"visited_vertices",
    x_field="x",
    y_field="y",
    z_field=None,
    coordinate_system='PROJCS["ETRS_1989_UWPP_1992",GEOGCS["GCS_ETRS_1989",DATUM["D_ETRS_1989",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Gauss_Kruger"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",-5300000.0],PARAMETER["Central_Meridian",19.0],PARAMETER["Scale_Factor",0.9993],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]];-5119200 -15295100 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision'
)

<Result 'D:\\sem5\\pag\\lab3\\lab3\\lab3.gdb\\visited_vertices'>

### Display roads

In [49]:
arcpy.analysis.Select(
    in_features="SKJZ",
    out_feature_class=r"D:\sem5\pag\lab3\lab3\lab3.gdb\visited_roads",
    where_clause=sql_expression
)


<Result 'D:\\sem5\\pag\\lab3\\lab3\\lab3.gdb\\SKJZ_Select'>